In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader



# Transform data to tensors and normalize
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_cifar_dataset(dataset_name):
    if dataset_name == "cifar10":
        trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
        testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
    elif dataset_name == "cifar100":
        (trainset, train_labels), (testset, test_labels) = tf.keras.datasets.cifar100.load_data()
    else:
        raise ValueError("Invalid dataset name. Choose either 'cifar10' or 'cifar100'.")

    # trainset, valset = train_test_split(trainset, test_size=0.2, random_state=42)
    
    return trainset, testset


2024-03-25 14:05:12.514580: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 14:05:12.514715: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 14:05:12.665104: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda:0


In [15]:
def train_model(model, criterion, optimizer, trainloader, device, valloader, num_epochs=10):
    for epoch in range(num_epochs):  # Number of epochs
        running_loss = 0.0
        with tqdm(total=len(trainloader), desc=f"Epoch {epoch+1}/{num_epochs}", unit='batch') as pbar:
            
            for inputs, labels in trainloader:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()

                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()
                pbar.update(1)
        # for inputs, labels in trainloader:
        #     optimizer.zero_grad()

        #     outputs = model(inputs)
        #     loss = criterion(outputs, labels)
        #     loss.backward()
        #     optimizer.step()

        #     running_loss += loss.item()
        print(f"Epoch [{epoch} | {num_epochs}], Training Loss: {running_loss / len(trainloader):.4f}")

        with torch.no_grad():
            val_loss = 0
            acc = 0
            for inputs, labels in valloader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                predicted = torch.argmax(outputs, dim = 1)
                acc += len(predicted[predicted == labels]) / len(labels)
                val_loss += loss.item()
        print(f"Epoch [{epoch} | {num_epochs}], Validation Loss: {val_loss / len(valloader):.4f}, Accuracy: {acc / len(valloader):.4f}")

        
    


def test_model(model, testloader, device):
    model.eval()
    model.to(device)
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(device), images.to(device)
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Accuracy on test set: {correct / total * 100:.2f}%")



In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = x.to(torch.float32)  
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x



In [5]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 8 * 8)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x



In [10]:
batch_size = 4
num_epochs = 5
num_workers = 2

dataset_name = "cifar10"

trainset, testset = load_cifar_dataset(dataset_name)
trainset, valset = train_test_split(trainset, test_size=0.2, random_state=42)
trainloader = DataLoader(trainset, batch_size=32, shuffle=True)
valloader = DataLoader(valset, batch_size=32, shuffle=True)
testloader = DataLoader(testset, batch_size=32, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


# MLP

In [59]:
input_size = 32 * 32 * 3
hidden_size1 = 512
hidden_size2 = 256
num_classes = 10

model = MLP(input_size, hidden_size1, hidden_size2, num_classes).to(device)
epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, criterion, optimizer, trainloader, device, valloader, num_epochs=epochs)



Epoch [0 | 10], Training Loss: 1.6796

Epoch [0 | 10], Validation Loss: 1.5803, Accuracy: 0.4394

Epoch [1 | 10], Training Loss: 1.4802

Epoch [1 | 10], Validation Loss: 1.4927, Accuracy: 0.4715

Epoch [2 | 10], Training Loss: 1.3729

Epoch [2 | 10], Validation Loss: 1.4606, Accuracy: 0.4901

Epoch [3 | 10], Training Loss: 1.2921

Epoch [3 | 10], Validation Loss: 1.4572, Accuracy: 0.4957

Epoch [4 | 10], Training Loss: 1.2039

Epoch [4 | 10], Validation Loss: 1.4337, Accuracy: 0.5007

Epoch [5 | 10], Training Loss: 1.1302

Epoch [5 | 10], Validation Loss: 1.4754, Accuracy: 0.5092

Epoch [6 | 10], Training Loss: 1.0581

Epoch [6 | 10], Validation Loss: 1.5367, Accuracy: 0.4997

Epoch [7 | 10], Training Loss: 0.9887

Epoch [7 | 10], Validation Loss: 1.5478, Accuracy: 0.5038

Epoch [8 | 10], Training Loss: 0.9191

Epoch [8 | 10], Validation Loss: 1.5882, Accuracy: 0.5085

Epoch [9 | 10], Training Loss: 0.8608

Epoch [9 | 10], Validation Loss: 1.6633, Accuracy: 0.5030


In [60]:
model = CNN(num_classes).to(device)
epochs = 10

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

train_model(model, criterion, optimizer, trainloader, device, valloader, num_epochs=epochs)



Epoch [0 | 10], Training Loss: 1.2900

Epoch [0 | 10], Validation Loss: 0.9865, Accuracy: 0.6551

Epoch [1 | 10], Training Loss: 0.8892

Epoch [1 | 10], Validation Loss: 0.8925, Accuracy: 0.6952

Epoch [2 | 10], Training Loss: 0.6719

Epoch [2 | 10], Validation Loss: 0.8269, Accuracy: 0.7168

Epoch [3 | 10], Training Loss: 0.4773

Epoch [3 | 10], Validation Loss: 0.8614, Accuracy: 0.7259

Epoch [4 | 10], Training Loss: 0.3011

Epoch [4 | 10], Validation Loss: 1.0001, Accuracy: 0.7111

Epoch [5 | 10], Training Loss: 0.1716

Epoch [5 | 10], Validation Loss: 1.1576, Accuracy: 0.7232

Epoch [6 | 10], Training Loss: 0.1229

Epoch [6 | 10], Validation Loss: 1.3097, Accuracy: 0.7156

Epoch [7 | 10], Training Loss: 0.0895

Epoch [7 | 10], Validation Loss: 1.4488, Accuracy: 0.7151

Epoch [8 | 10], Training Loss: 0.0791

Epoch [8 | 10], Validation Loss: 1.5372, Accuracy: 0.7174

Epoch [9 | 10], Training Loss: 0.0663

Epoch [9 | 10], Validation Loss: 1.6297, Accuracy: 0.7148


In [6]:
from torchvision.models import vgg16, vgg19


In [17]:
vgg_model = vgg19(pretrained=True).to(device)
# Freeze all layers
for param in vgg_model.parameters():
    param.requires_grad = False
    
for param in vgg_model.features.parameters():
    param.requires_grad = True

num_features = vgg_model.classifier[6].in_features
vgg_model.classifier[6] = nn.Linear(num_features, 10)  # Assuming 10 classes for CIFAR-10
vgg_model = vgg_model.to(device)
epochs = 10 

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg_model.parameters(), lr=0.001, momentum=0.9)

train_model(vgg_model, criterion, optimizer, trainloader, device, valloader, num_epochs=epochs)



Epoch 1/10: 100%|██████████| 1250/1250 [00:40<00:00, 30.51batch/s]


Epoch [0 | 10], Training Loss: 0.7322
Epoch [0 | 10], Validation Loss: 0.5711, Accuracy: 0.8069


Epoch 2/10: 100%|██████████| 1250/1250 [00:39<00:00, 31.44batch/s]


Epoch [1 | 10], Training Loss: 0.4047
Epoch [1 | 10], Validation Loss: 0.4500, Accuracy: 0.8550


Epoch 3/10: 100%|██████████| 1250/1250 [00:40<00:00, 31.08batch/s]


...

Epoch [7 | 10], Training Loss: 0.0670
Epoch [7 | 10], Validation Loss: 0.4622, Accuracy: 0.8842


Epoch 9/10: 100%|██████████| 1250/1250 [00:40<00:00, 31.12batch/s]


Epoch [8 | 10], Training Loss: 0.0443
Epoch [8 | 10], Validation Loss: 0.4971, Accuracy: 0.8838


Epoch 10/10: 100%|██████████| 1250/1250 [00:40<00:00, 31.11batch/s]


Epoch [9 | 10], Training Loss: 0.0425
Epoch [9 | 10], Validation Loss: 0.5497, Accuracy: 0.8811
